In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash


# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Ryan Hubbuck Dashboard')

image_filename = 'grazioso-logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.A([
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
               height = 200, width = 200))], href = 'https://www.snhu.edu', target = "_blank"),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Ryan Hubbuck'))),
    html.Hr(),
    html.Div(      
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        ) 
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=True,
        row_selectable="single", #allow a row to be selected
        selected_rows=[],
        filter_action="native", #allow a filter
        sort_action="native", #allow sorting
        page_action="native", #enable pagination
        page_current=0, #set start page
        page_size=10, #set rows per page
    ),
    html.Br(),
     html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
     html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
        # Water Rescue filter
        if filter_type == 'water':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
        # Mountain or Wilderness Rescue filter
        elif filter_type == 'mountain':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        # Disaster Rescue or Individual Tracking filter
        elif filter_type == 'disaster':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            
        # Reset to no filter
        else:
            df = pd.DataFrame.from_records(shelter.read({}))
            
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        return (data,columns)

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    dffPie = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.pie(dffPie, names='breed', title='Preferred Animals')
        )    
    ]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):    
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
        center=[30.75,-97.48], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
            children=[
            dl.Tooltip(dff.iloc[row,4]),
            dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[row,9])
            ])
        ])
    ])
]

app.run_server(debug=True)

OSError: Address 'http://127.0.0.1:26551' already in use.
    Try passing a different port to run_server.

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/usr/local/anaconda/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda/lib/python3.9/site-packages/retrying.py", line 49, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
  File "/usr/local/anaconda/lib/python3.9/site-packages/retrying.py", line 212, in call
    raise attempt.get()
  File "/usr/local/anaconda/lib/python3.9/site-packages/retrying.py", line 247, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "/usr/local/anaconda/lib/python3.9/site-packages/six.py", line 719, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.9/site-packages/retrying.py", line 200, in call
    attempt = Attempt(fn(*args, **kwargs), attempt_number, False)
  File "/usr/local/anaconda/lib/python3.9/site-pa